<div style="width:100%; background-color: #D9EDF7; border: 1px solid #CFCFCF; text-align: left; padding: 10px;">
      <b>Renewable power plants: Validation and output notebook</b>
      <ul>
        <li><a href="main.ipynb">Main notebook</a></li>
        <li><a href="download_and_process.ipynb">Download and process notebook</a></li>
        <li>Validation and output notebook</li>
      </ul>
      <br>This notebook is part of the <a href="http://data.open-power-system-data.org/renewable_power_plants"> Renewable power plants Data Package</a> of <a href="http://open-power-system-data.org">Open Power System Data</a>.
</div>

Part 1 of the script (<a href="download_and_process.ipynb">Download and process Notebook</a>) has downloaded and merged the original data. This Notebook subsequently checks, validates the list of renewable power plants and creates CSV/XLSX/SQLite files. It also generates a daily time series of cumulated installed capacities by energy source.

*(Before running this script make sure you ran Part 1, so that the renewables.pickle files for each country exist in the same folder as the scripts)*


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Initialization" data-toc-modified-id="Initialization-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initialization</a></span><ul class="toc-item"><li><span><a href="#Script-setup" data-toc-modified-id="Script-setup-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Script setup</a></span></li><li><span><a href="#Load-data" data-toc-modified-id="Load-data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Load data</a></span></li></ul></li><li><span><a href="#Validation-Markers" data-toc-modified-id="Validation-Markers-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Validation Markers</a></span><ul class="toc-item"><li><span><a href="#Germany-DE" data-toc-modified-id="Germany-DE-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Germany DE</a></span></li><li><span><a href="#France-FR" data-toc-modified-id="France-FR-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>France FR</a></span></li><li><span><a href="#United-Kingdom-UK" data-toc-modified-id="United-Kingdom-UK-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>United Kingdom UK</a></span></li></ul></li><li><span><a href="#Harmonization" data-toc-modified-id="Harmonization-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Harmonization</a></span><ul class="toc-item"><li><span><a href="#Harmonizing-column-order" data-toc-modified-id="Harmonizing-column-order-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Harmonizing column order</a></span></li><li><span><a href="#Cleaning-fields" data-toc-modified-id="Cleaning-fields-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Cleaning fields</a></span></li><li><span><a href="#Sort" data-toc-modified-id="Sort-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Sort</a></span></li><li><span><a href="#Leave-unspecified-cells-blank" data-toc-modified-id="Leave-unspecified-cells-blank-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Leave unspecified cells blank</a></span></li><li><span><a href="#Separate-dirty-from-clean" data-toc-modified-id="Separate-dirty-from-clean-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Separate dirty from clean</a></span></li></ul></li><li><span><a href="#Capacity-time-series" data-toc-modified-id="Capacity-time-series-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Capacity time series</a></span><ul class="toc-item"><li><span><a href="#Germany" data-toc-modified-id="Germany-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Germany</a></span></li><li><span><a href="#United-Kingdom" data-toc-modified-id="United-Kingdom-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>United Kingdom</a></span></li></ul></li><li><span><a href="#Output" data-toc-modified-id="Output-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Output</a></span><ul class="toc-item"><li><span><a href="#Write-data-files" data-toc-modified-id="Write-data-files-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Write data files</a></span><ul class="toc-item"><li><span><a href="#Write-CSV-files" data-toc-modified-id="Write-CSV-files-5.1.1"><span class="toc-item-num">5.1.1&nbsp;&nbsp;</span>Write CSV-files</a></span></li><li><span><a href="#Write-XLSX-files" data-toc-modified-id="Write-XLSX-files-5.1.2"><span class="toc-item-num">5.1.2&nbsp;&nbsp;</span>Write XLSX-files</a></span></li></ul></li><li><span><a href="#Write-meta-data" data-toc-modified-id="Write-meta-data-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Write meta data</a></span></li><li><span><a href="#Generate-checksums" data-toc-modified-id="Generate-checksums-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Generate checksums</a></span></li></ul></li></ul></div>

# Initialization

In [ ]:
settings = {
    'version': '2019-02-25',
    'changes': 'Updated all countries with new data available (DE, FR, CH, DK) and added data for UK.'
}

## Script setup

In [ ]:
import json
import logging
import os
import urllib.parse
import re
import zipfile

import pandas as pd
import numpy as np
import requests
import sqlalchemy
import yaml
import hashlib
import os
import fiona
import cartopy.io.shapereader as shpreader
import shapely.geometry as sgeom
from shapely.prepared import prep
from shapely.ops import unary_union
import fake_useragent

%matplotlib inline

# Option to make pandas display 40 columns max per dataframe (default is 20)
pd.options.display.max_columns = 40

# Create input and output folders if they don't exist
os.makedirs(os.path.join('input', 'original_data'), exist_ok=True)

os.makedirs('output', exist_ok=True)
os.makedirs(os.path.join('output', 'renewable_power_plants'), exist_ok=True)
package_path = os.path.join('output', 'renewable_power_plants',settings['version'])
os.makedirs(package_path, exist_ok=True)

## Load data

In [ ]:
countries = set(['DE', 'DK','FR','PL','CH', 'UK'])
countries_non_DE = countries - set(['DE'])
countries_dirty = set(['DE_outvalidated_plants', 'FR_overseas_territories'])
countries_including_dirty = countries | countries_dirty

# Read data from script Part 1 download_and_process
dfs = {}
for country in countries:
    dfs[country] = pd.read_pickle('intermediate/'+country+'_renewables.pickle')

## Download coastline data

The coastline shapefile is needed to check if the geocoordinates of the land powerplants point to a land location, and conversely, if the geocoordinates of the onshore facilities point to a location not on land.

In [ ]:
coastline_url = 'https://www.ngdc.noaa.gov/mgg/shorelines/data/gshhg/latest/gshhg-shp-2.3.7.zip'

user_agent = fake_useragent.UserAgent()

directory_path = os.path.join('input', 'maps', 'coastline')
os.makedirs(directory_path, exist_ok=True)
filepath = os.path.join(directory_path, 'gshhg-shp-2.3.7.zip')

# check if the file exists; if not, download it
if not os.path.exists(filepath):
    session = requests.session()
    print(coastline_url)
    print('Downloading...')
    headers = {'User-Agent' : user_agent.random}
    r = session.get(coastline_url, headers=headers, stream=True)
    total_size = r.headers.get('content-length')
    total_size = int(total_size)
    chuncksize = 4096
    with open(filepath, 'wb') as file:
        downloaded = 0
        for chunck in r.iter_content(chuncksize):
            file.write(chunck)
            downloaded += chuncksize
            print('\rProgress: {:.2f}%'.format(100 * downloaded / float(total_size)), end='')
    print(' Done.')
    zip_ref = zipfile.ZipFile(filepath, 'r')
    zip_ref.extractall(directory_path)
    zip_ref.close()
else:
    print('The file is already there:', filepath)
    filepath = '' + filepath

coastline_shapefile_path = os.path.join('input', 'maps', 'coastline', 'GSHHS_shp', 'f', 'GSHHS_f_L1.shp')
print("Shapefile path: ", coastline_shapefile_path)

### Load the list of sources

In [ ]:
source_df = pd.read_csv(os.path.join('input', 'sources.csv'))
source_df

# Validation Markers

This section checks the DataFrame for a set of pre-defined criteria and adds markers to the entries in an additional column. The marked data will be included in the output files, but marked, so that they can be easiliy filtered out. For creating the validation plots and the time series, suspect data is skipped.

In [ ]:
validation_marker = {}
mark_rows = {}

## Germany DE

**Add marker to data according to criteria (see validation_marker above)**

In [ ]:
# It seems that some DE.commissioning_date values are integers, which causes
# the parts of the code dealing with dates to break.
integer_dates_mask = dfs['DE'].apply(lambda row: type(row['commissioning_date']) is int, axis=1).values

print("Integer dates")
display(dfs['DE'][integer_dates_mask])

dfs['DE']=dfs['DE'][~integer_dates_mask]
dfs['DE'].reset_index(drop=True, inplace=True)

In [ ]:
key = 'R_1'
cutoff_date_bnetza = '2017-12-31'
cutoff_date_bnetza = pd.Timestamp(2017, 12, 31)

mark_rows[key] = (dfs['DE']['commissioning_date'] <= cutoff_date_bnetza) &\
                 (dfs['DE']['data_source'].isin(['BNetzA', 'BNetzA_PV', 'BNetzA_PV_historic']))

#mark_rows[key] = (
#                    (dfs['DE']['commissioning_date'] <= cutoff_date_bnetza) 
#                    & (dfs['DE']['data_source'] == 'BNetzA')
#                 ) | (
#                    (dfs['DE']['commissioning_date'] <= cutoff_date_bnetza)
#                    & (dfs['DE']['data_source'] == 'BNetzA_PV')
#                 ) | (
#                    (dfs['DE']['commissioning_date'] <= cutoff_date_bnetza)
#                    & (dfs['DE']['data_source'] == 'BNetzA_PV_historic')
#                 )
validation_marker[key] = {
    "Short explanation": "data_source = BNetzA and commissioning_date < " + str(cutoff_date_bnetza.date()),
    "Long explanation": "This powerplant is probably also represented by an entry from the TSO data and should therefore be filtered out."
}

In [ ]:
key = 'R_2'
mark_rows[key] = ((dfs['DE']['notification_reason'] != 'Inbetriebnahme') &
                 (dfs['DE']['data_source'] == 'BNetzA'))
validation_marker[key] = {
    "Short explanation": "notification_reason other than commissioning (Inbetriebnahme)",
    "Long explanation": "This powerplant is probably represented by an earlier entry already (possibly also from the TSO data) and should therefore be filtered out."
}

key = 'R_3'
mark_rows[key] = (dfs['DE']['commissioning_date'].isnull())
validation_marker[key] = {
    "Short explanation": "commissioning_date not specified",
    "Long explanation": ""
}

key = 'R_4'
mark_rows[key] = dfs['DE'].electrical_capacity <= 0.0
validation_marker[key] = {
    "Short explanation": "electrical_capacity not specified",
    "Long explanation": ""
}

key = 'R_5'
mark_rows[key] = dfs['DE']['grid_decommissioning_date'].isnull() == False # Just the entry which is not double should be kept, thus the other one is marked
validation_marker[key] = {
    "Short explanation": "decommissioned from the grid",
    "Long explanation": "This powerplant is probably commissioned again to the grid of another grid operator and therefore this doubled entry should be filtered out."
}

key = 'R_6'
mark_rows[key] = dfs['DE']['decommissioning_date'].isnull() == False
validation_marker[key] = {
    "Short explanation": "decommissioned",
    "Long explanation": "This powerplant is completely decommissioned."
}

key = 'R_8' # note that we skip R7 here as R7 is used for frech oversees power plants below (we never change meanings of R markers, so R7 stays reserved for that)
mark_rows[key] = (dfs['DE'].duplicated(['eeg_id'],keep='first') # note that this depends on BNetzA items to be last in list, because we want to keep the TSO items
                  & (dfs['DE']['eeg_id'].isnull() == False)) 
validation_marker[key] = {
    "Short explanation": "duplicate_eeg_id",
    "Long explanation": "This power plant is twice in the data (e.g. through BNetzA and TSOs)."
}

In [ ]:
dfs['DE']['comment'] = ''
for key, rows_to_mark in mark_rows.items():
    dfs['DE'].loc[rows_to_mark, 'comment'] += key+"|"

del mark_rows, key, rows_to_mark # free variables no longer needed

In [ ]:
# Summarize capacity of suspect data by data_source
display(dfs['DE'].groupby(['comment', 'data_source'])['electrical_capacity'].sum().to_frame())

# Summarize capacity of suspect data by energy source
dfs['DE'].groupby(['comment', 'energy_source_level_2'])['electrical_capacity'].sum().to_frame()

**Create cleaned DataFrame**

All marked entries are deleted for the cleaned version of the DataFrame that is utilized for creating time series of installation and for the validation plots.

## France FR

In [ ]:
# Create empty marker column
dfs['FR']['comment'] = ""

key = 'R_7'
mark_rows_FR_not_in_Europe = dfs['FR'][((dfs['FR']['lat'] < 41) |
                       (dfs['FR']['lon'] < -6) |
                       (dfs['FR']['lon'] > 10))].index
validation_marker[key] = {
    "Short explanation": "not connected to the European grid",
    "Long explanation": "This powerplant is located in regions belonging to France but not located in Europe (e.g. Guadeloupe)."
}

dfs['FR'].loc[mark_rows_FR_not_in_Europe, 'comment'] += key+"|"

del mark_rows_FR_not_in_Europe

## United Kingdom UK

In [ ]:
# Create an empty marker column
dfs['UK']['comment'] = ""

# Create a function to check if an offshore powerplant is not wind
geoms = fiona.open(coastline_shapefile_path)
land_geom = sgeom.MultiPolygon([sgeom.shape(geom['geometry']) for geom in geoms])
land = prep(land_geom)

def not_on_land_but_should_be(powerplant_data):
    longitude = powerplant_data['lon']
    latitude = powerplant_data['lat']
    if pd.isnull(longitude) or pd.isnull(latitude):
        return False
    not_on_land = not land.contains(sgeom.Point(longitude, latitude))
    offshore_ok =  'Offshore' in [powerplant_data['region'], powerplant_data['municipality']] or \
                  (powerplant_data['energy_source_level_2'] in ['Wind', 'Marine'])
    return not_on_land and not offshore_ok
    

key = 'R_9'
validation_marker[key] = {
    "Short explanation": "Not on land, but should be.",
    "Long explanation": "The geocoordinates of this powerplant indicate that it is not on the UK mainland, but the facility is not an offshore wind farm."
}

mark_rows_UK_not_on_land = dfs['UK'].apply(lambda row: not_on_land_but_should_be(row), axis=1)
dfs['UK'].loc[mark_rows_UK_not_on_land, 'comment'] += key+"|"

del mark_rows_UK_not_on_land

# Harmonization

## Harmonizing column order

In [ ]:
field_lists = {
    'DE': ['commissioning_date', 'decommissioning_date', 'energy_source_level_1', 'energy_source_level_2','energy_source_level_3', 'technology',
           'electrical_capacity', 'thermal_capacity', 'voltage_level', 'tso', 'dso', 'dso_id', 'eeg_id', 'bnetza_id',
           'federal_state', 'postcode', 'municipality_code', 'municipality', 'address', 'address_number',
           'utm_zone', 'utm_east', 'utm_north', 'lat', 'lon', 'data_source', 'comment'],
    'DK': ['commissioning_date', 'energy_source_level_1', 'energy_source_level_2', 'energy_source_level_3', 'technology', 'electrical_capacity',
           'dso', 'gsrn_id', 'postcode', 'municipality_code', 'municipality', 'address', 'address_number',
           'utm_east', 'utm_north', 'lat', 'lon', 'hub_height', 'rotor_diameter', 'manufacturer', 'model', 'data_source'],
    'FR': ['municipality_code', 'municipality', 'energy_source_level_1', 'energy_source_level_2', 'energy_source_level_3', 
           'technology', 'electrical_capacity', 'number_of_installations', 'lat', 'lon', 'data_source', 'as_of_year', 'comment'],
    'PL': ['district', 'energy_source_level_1', 'energy_source_level_2', 'energy_source_level_3', 'technology',
           'electrical_capacity', 'number_of_installations', 'as_of_year', 'data_source'],
    'CH': ['commissioning_date', 'municipality', 'energy_source_level_1', 'energy_source_level_2', 'energy_source_level_3',
           'technology','electrical_capacity', 'municipality_code', 'project_name', 'production', 'tariff',
           'contract_period_end', 'street', 'canton', 'company', 'lat', 'lon', 'data_source'],
    'UK': ['commissioning_date', 'uk_beis_id', 'site_name', 'operator',
           'energy_source_level_1', 'energy_source_level_2', 'energy_source_level_3', 'technology',
           'electrical_capacity', 'chp', 'support_robranding', 'support_fit', 'support_cfd', 'capacity_individual_turbine',
           'number_of_turbines', 'solar_mounting_type', 'address', 'municipality', 'region', 'country', 'postcode',
           'lat', 'lon', 'data_source', 'comment']
}

for country in field_lists:
    for c in field_lists[country]:
        if c not in dfs[country].columns:
            print(country, c)
    dfs[country] = dfs[country].loc[:, field_lists[country]]

## Cleaning fields

Five digits behind the decimal point for decimal fields. Dates should be without timestamp.

In [ ]:
cleaning_specs = {
    'decimal' : {
        'DE': ['electrical_capacity','lat','lon','utm_east','utm_north','thermal_capacity'],
        'DK': ['electrical_capacity','lat','lon','utm_east','utm_north'],
        'CH': ['electrical_capacity','lat','lon'],
        'FR': ['electrical_capacity','lat','lon'],
        'PL': ['electrical_capacity'],
        'UK': ['electrical_capacity', 'lat', 'lon']
    },
    'integer': {
        'DE': ['utm_zone'],
        'UK': ['uk_beis_id']
    },
    'date': {
        'DE': ['commissioning_date', 'decommissioning_date'],
        'DK': ['commissioning_date'],
        'CH': ['commissioning_date'],
        'UK': ['commissioning_date']
    },    
}

for cleaning_type, cleaning_spec in cleaning_specs.items():
    for country, fields in cleaning_spec.items():
        for field in fields:
            print('Cleaning ' + country + '.' + field +' to ' + cleaning_type + '.')
            if cleaning_type == 'decimal':
                dfs[country][field] = dfs[country][field].map(lambda x: round(x, 8))
            elif cleaning_type == 'integer':
                dfs[country][field] = pd.to_numeric(dfs[country][field], errors='coerce')
                dfs[country][field] = dfs[country][field].map(lambda x: '%.0f' % x)  
            elif cleaning_type == 'date':
                dfs[country][field] = dfs[country][field].apply(lambda x: x.date())

print("Done!")

del cleaning_specs

## Sort

In [ ]:
sort_by = {
    'DE': 'commissioning_date',
    'DK': 'commissioning_date',
    'CH': 'commissioning_date',
    'FR': 'municipality_code',
    'PL': 'district',
    'UK': 'commissioning_date'
}

for country, sort_by in sort_by.items():
    print('Sorting', country)
    try:
        dfs[country] = dfs[country].iloc[dfs[country][sort_by].sort_values().index]
    except Exception as e:
        print('\tException:',e)
print('Done!')
del sort_by

## Leave unspecified cells blank

This step may take some time.

In [ ]:
for country in countries:
    print(country)
    dfs[country].fillna('', inplace=True)

## Separate dirty from clean

We separate all plants which have a validation marker in the comments column into a separate DataFrame and eventually also in a separate CSV file, so the main country files only contain "clean" plants, i.e. those without any special comment. This is useful since all our comments denote that most people would probably not like to include them in their calculations.

In [ ]:
dirty_keys = {
    'DE' : 'DE_outvalidated_plants',
    'FR' : 'FR_overseas_territories',
} 

for country in dirty_keys.keys():
    print(country)
    idx_dirty = dfs[country][dfs[country].comment.str.len() > 1].index
    dirty_key = dirty_keys[country]
    dfs[dirty_key] = dfs[country].loc[idx_dirty]
    dfs[country] = dfs[country].drop(idx_dirty)

del idx_dirty, dirty_key

# Capacity time series

This section creates a daily and yearly time series of the cumulated installed capacity by energy source for the United Kingdom, Germany, Denmark, and Switzerland. Three time series will be created for the UK: one for the whole country (GB-UKM), one for Northern Ireland (GB-NIR), and one for the Great Britain (GB-GBN). This data will be part of the output and will be compared in a plot for validation in the next section.

In [ ]:
daily_timeseries = {}


# One DE row has has year > 2019, so I removed it here.
m=dfs['DE'].apply(lambda row: row['commissioning_date'].year > 2019, axis=1)
display(dfs['DE'][m])
dfs['DE'] = dfs['DE'][~m]

In [ ]:
def to_new_level(row):
    complete_level = ':'.join(row.values).replace('::',':').strip(':')
    return complete_level.replace('Other or unspecified technology', 'other').lower().replace(':', '_')

def to_daily_timeseries(df, start_date, end_date):  
    # Combine energy levels to most new standardized values
    df['energy_type'] = df[['energy_source_level_2', 'energy_source_level_3', 'technology']].apply(to_new_level, axis=1)
    
    # Set range of time series as index
    daily_timeseries = pd.DataFrame(index=pd.date_range(start=start_date, end=end_date, freq='D'))
    
    # Create cumulated time series per energy source for both yearly and daily time series
    for energy_type in df['energy_type'].unique():
        temp = (df[['commissioning_date', 'electrical_capacity']]
            .loc[df['energy_type'] == energy_type])
        temp_timeseries = temp.set_index('commissioning_date')
        temp_timeseries.index = pd.DatetimeIndex(temp_timeseries.index)

        # Create cumulated time series per energy_source and day
        daily_timeseries[energy_type] = temp_timeseries.resample('D').sum().cumsum().fillna(method='ffill')
        
        # Make sure that the columns are properly filled
        daily_timeseries[energy_type]= daily_timeseries[energy_type].fillna(method='ffill').fillna(value='')
    
    # Reset the time index
    daily_timeseries.reset_index(inplace=True)

    # Set the index name
    daily_timeseries.rename(columns={'index': 'day'}, inplace=True)
    
    # Drop the temporary column "energy_type"
    df.drop('energy_type', axis=1, inplace=True)
    return daily_timeseries

eligible_for_timeseries = [country for country in countries if 'commissioning_date' in dfs[country].columns]
#eligible_for_timeseries = ['CH', 'DK', 'UK', 'DE']
possible_start_dates = [dfs[country]['commissioning_date'].min() for country in eligible_for_timeseries]
possible_end_dates = [dfs[country]['commissioning_date'].max() for country in eligible_for_timeseries]

#print("Possible start and end dates:")
#for country in eligible_for_timeseries:
#    print(country, dfs[country]['commissioning_date'].min(), dfs[country]['commissioning_date'].max())

start_date = min(possible_start_dates)
end_date = max(possible_end_dates)

for country in eligible_for_timeseries:
    print("Timeseries for", country)
    try:
        daily_timeseries[country] = to_daily_timeseries(dfs[country], start_date, end_date)
        print('\t Done!')
    except Exception as e:
        print('\t', e)

## Make separate series for Great Britain and Northern Ireland

In [ ]:
# Create the mask for Norther Ireland
ni_mask = dfs['UK']['country'] == 'Northern Ireland'

# Split the UK data
ni_df = dfs['UK'][ni_mask].copy()
gb_df = dfs['UK'][~ni_mask].copy()

# Make the timeseries for Northern Ireland
daily_timeseries['GB-NIR'] = to_daily_timeseries(ni_df, start_date, end_date)

# Make the timeseries for Great Briatin (England, Wales, Scotland)
daily_timeseries['GB-GBN'] = to_daily_timeseries(gb_df, start_date, end_date)

# Renaming the entry for UK to conform to the ISO codes
daily_timeseries['GB-UKM'] = daily_timeseries.pop('UK')

## Create one time series for all the countries

In [ ]:
unified_daily_timeseries = pd.DataFrame(index=pd.date_range(start=start_date, end=end_date, freq='D'))
# Append the country name to capacity columns' names
for c in daily_timeseries:
    new_columns = [c + "__" + col if col != 'day' else 'day' for col in daily_timeseries[c].columns]
    daily_timeseries[c].columns = new_columns
    
# Unify separate series
unified_daily_timeseries = pd.concat(daily_timeseries.values(), axis=1, sort=False)

# Make sure the day column appears only one
days = unified_daily_timeseries['day']
unified_daily_timeseries.drop('day', axis=1, inplace=True)
unified_daily_timeseries['day'] = days.iloc[:, 0]

In [ ]:
# Show some rows
unified_daily_timeseries.tail(2)

# Make the normalized dataframe for all the countries

Here, we create a dataframe containing the following data for all the countries:


In [ ]:
geographical_resolution = {
    'PL' : 'district',
    'FR' : 'municipality',
    'CH' : 'municipality',
    'DE' : 'address',
    'DK' : 'address',
    'UK' : 'address'
}

dfs_to_concat = []

columns = ['energy_source_level_1', 'energy_source_level_2', 'energy_source_level_3', 'electrical_capacity',
           'data_source', 'municipality', 'lon', 'lat', 'commissioning_date', 'geographical_resolution', 
           'as_of_year'
          ]

for country in countries:
    country_df = dfs[country].loc[:, columns].copy()
    country_df['country'] = country
    country_df['geographical_resolution'] = geographical_resolution[country]
    if country == 'PL':
        country_df['as_of_year'] = 2016
    elif country == 'FR':
        country_df['as_of_year'] = 2017
    dfs_to_concat.append(country_df)

european_df = pd.concat(dfs_to_concat)
european_df.reset_index(inplace=True, drop=True)

european_df.sample(n=10)

# Output
This section finally writes the Data Package:
* CSV + XLSX + SQLite
* Meta data (JSON)

In [ ]:
os.makedirs(package_path, exist_ok=True)

## Write data files

### Write CSV-files

One csv-file for each country. This process will take some time depending on you hardware.

In [ ]:
# Write each country's dataset as a separate csv file
table_names = {}
for country in countries_including_dirty:
    print(country)
    table_names[country] = 'renewable_power_plants_'+country if country not in countries_dirty else 'res_plants_separated_'+country
    dfs[country].to_csv(os.path.join(package_path, table_names[country]+'.csv'),
            sep=',',
            decimal='.',
            date_format='%Y-%m-%d',
            line_terminator='\n',
            encoding='utf-8',
            index=False)

In [ ]:
# Write daily cumulated time series as csv
filename = 'renewable_capacity_timeseries.csv'.format(country)
print(filename)
unified_daily_timeseries.to_csv(os.path.join(package_path, filename),
        sep=',',
        float_format='%.3f',
        decimal='.',
        date_format='%Y-%m-%d',
        encoding='utf-8',
        index=False)

In [ ]:
european_df.to_csv(os.path.join(package_path, 'renewable_power_plants_eu.csv'),
            sep=',',
            decimal='.',
            date_format='%Y-%m-%d',
            line_terminator='\n',
            encoding='utf-8',
            index=False)

In [ ]:
# Write csv of Marker Explanations
validation_marker_df = pd.DataFrame(validation_marker).transpose()
validation_marker_df = validation_marker_df.iloc[:, ::-1] # Reverse column order
validation_marker_df.index.name = 'Validation marker'
validation_marker_df.reset_index(inplace=True)
validation_marker_df.to_csv(os.path.join(package_path, 'validation_marker.csv'), 
        sep=',',
        decimal='.',
        date_format='%Y-%m-%d',
        line_terminator='\n',
        encoding='utf-8',
        index=False)

### Write XLSX-files

This process will take some time depending on your hardware.

All country power plant list will be written in one xlsx-file. Each country power plant list is written in a separate sheet. As the German power plant list has too many entries for one sheet, it will be split in two. An additional sheet includes the explanations of the marker.

In [ ]:
# Write the results as xlsx file
%time writer = pd.ExcelWriter(os.path.join(package_path, 'renewable_power_plants.xlsx'), engine='xlsxwriter', date_format='yyyy-mm-dd')

In [ ]:
print('Writing DE part 1')
%time dfs['DE'][:1000000].to_excel(writer, index=False, sheet_name='DE part-1')

print('Writing DE part 2')
%time dfs['DE'][1000000:].to_excel(writer, index=False, sheet_name='DE part-2')

In [ ]:
display(dfs.keys())
u=u
for country in (countries_non_DE | countries_dirty):
    print('Writing ' + country)
    %time dfs[country].to_excel(writer, index=False, sheet_name=country)

print('Writing validation marker sheet')
%time validation_marker_df.to_excel(writer, index=False, sheet_name='validation_marker')

# Save timeseries as Excel
%time unified_daily_timeseries[country].to_excel(writer, index=False, sheet_name='capacity_timeseries')

print('Saving...')
%time writer.save()
print('...done!')

**Write SQLite**

In [ ]:
# Some date columns are giving the engine some trouble, therefore cast to string: 
if 'DE' in dfs:
    dfs['DE'].decommissioning_date = dfs['DE'].decommissioning_date.astype(str)
    dfs['DE'].commissioning_date = dfs['DE'].commissioning_date.astype(str)
    dfs['DE_outvalidated_plants'].commissioning_date = dfs['DE_outvalidated_plants'].commissioning_date.astype(str)

# Using chunksize parameter is for lower
# memory computers. Removing it might speed things up.
engine = sqlalchemy.create_engine('sqlite:///' + package_path + '/renewable_power_plants.sqlite')  

for country in countries_including_dirty:
    if country=='DE_outvalidated_plants':
        continue # The DE_outvalidated_plants file gives a strange error message. Therefore do not put it into SQLite.
    %time dfs[country].to_sql(table_names[country],engine,if_exists="replace",chunksize=100000,index=False)

validation_marker_df.to_sql('validation_marker', engine, if_exists="replace", chunksize=100000, index=False)

In [ ]:
# Save timeseries as sqlite
%time unified_daily_timeseries[country].to_sql('renewable_capacity_timeseries',
                                               engine,
                                               if_exists="replace",
                                               chunksize=100000,
                                               index=False)

## Write meta data

The Data Packages meta data are created in the specific JSON format as proposed by the Open Knowledge Foundation. Please see the Frictionless Data project by OKFN (http://data.okfn.org/) and the Data Package specifications (http://dataprotocols.org/data-packages/) for more details.

In order to keep the Jupyter Notebook more readable the metadata is written in the human-readable YAML format using a multi-line string and then parse the string into a Python dictionary and save it as a JSON file.

In [ ]:
metadata = """
hide: yes
name: opsd-renewable-power-plants
title: Renewable power plants
description: List of renewable energy power stations
long_description: >-
    This Data Package contains a list of renewable energy power plants in lists of 
    renewable energy-based power plants of Germany, Denmark, France and Poland. 
    Germany: More than 1.7 million renewable power plant entries, eligible under the 
    renewable support scheme (EEG). 
    Denmark: Wind and phovoltaic power plants with a high level of detail. 
    France: Aggregated capacity and number of installations per energy source per 
    municipality (Commune). 
    Poland: Summed capacity and number of installations per energy source 
    per municipality (Powiat).
    Switzerland: Renewable power plants eligible under the Swiss feed in tariff KEV 
    (Kostendeckende Einspeisevergütung).
    United Kingdom: Renewable power plants in the United Kingdom.
    Due to different data availability, the power plant lists are of different 
    accurancy and partly provide different power plant parameter. Due to that, the 
    lists are provided as seperate csv-files per country and as separate sheets in the
    excel file. Suspect data or entries with high probability of duplication are marked
    in the column 'comment'. Theses validation markers are explained in the file
    validation_marker.csv. Filtering all entries with comments out results in the recommended
    data set.
    Additionally, the Data Package includes daily time series of cumulated
    installed capacity per energy source type for Germany, Denmark, Switzerland and the United Kingdom. All data processing is 
    conducted in Python and pandas and has been documented in the Jupyter Notebooks 
    linked below. 
keywords: [master data register,power plants,renewables,germany,denmark,france,poland,switzerland,united kingdom,open power system data]
geographical-scope: Germany, Denmark, France, Poland, Switzerland, United Kingdom
resources:
    - path: renewable_power_plants_DE.csv
      format: csv
      encoding: UTF-8
      missingValue: ""
      schema:         
          fields:
            - name: commissioning_date
              type: date
              format: YYYY-MM-DD
              description: Date of commissioning of specific unit
              opsd-contentfilter: "true"
            - name: decommissioning_date
              type: date
              format: YYYY-MM-DD
              description: Date of decommissioning of specific unit
            - name: energy_source_level_1
              description: Type of energy source (e.g. Renewable energy)
              type: string
            - name: energy_source_level_2
              description: Type of energy source (e.g. Wind, Solar)
              type: string
              opsd-contentfilter: "true"
            - name: energy_source_level_3
              description: Subtype of energy source (e.g. Biomass and biogas)
              type: string
            - name: technology
              description: Technology to harvest energy source (e.g. Onshore, Photovoltaics)
              type: string
            - name: electrical_capacity
              description: Installed electrical capacity in MW
              type: number
              format: float
              unit: MW
            - name: thermal_capacity
              description: Installed thermal capacity in MW
              type: number
              format: float
              unit: MW
            - name: voltage_level
              description: Voltage level of grid connection
              type: string
            - name: tso
              description: Name of transmission system operator of the area the plant is located
              type: string
            - name: dso
              description: Name of distribution system operator of the region the plant is located in
              type: string
            - name: dso_id
              description: Company number of German distribution grid operator
              type: string
            - name: eeg_id
              description: Power plant EEG (German feed-in tariff law) remuneration number
              type: string
            - name: bnetza_id
              description: Power plant identification number by BNetzA
              type: string
            - name: federal_state
              description: Name of German administrative level 'Bundesland'
              type: string
            - name: postcode
              description: German zip-code
              type: string
            - name: municipality_code
              description: German Gemeindenummer (municipalitiy number)
              type: string
            - name: municipality
              description: Name of German Gemeinde (municipality)
              type: string
            - name: address
              description: Street name or name of land parcel
              type: string
            - name: address_number
              description: House number or number of land parcel
              type: string
            - name: utm_zone
              description: Universal Transverse Mercator zone value
              type: 
            - name: utm_east
              description: Coordinate in Universal Transverse Mercator (east)
              type: numeric
              format: float
            - name: utm_north
              description: Coordinate in Universal Transverse Mercator (north)
              type: numeric
              format: float
            - name: lat
              description: Latitude coordinates
              type: geopoint
              format: lat
            - name: lon
              description: Longitude coordinates 
              type: geopoint
              format: lon
            - name: data_source
              description: Source of database entry
              type: string
            - name: comment
              description: Shortcodes for comments related to this entry, explanation can be looked up in validation_marker.csv
              type: string
    - path: renewable_power_plants_DK.csv
      format: csv
      encoding: UTF-8
      missingValue: ""
      schema:
          fields:
            - name: commissioning_date
              type: date
              format: YYYY-MM-DD
              opsd-contentfilter: "true"
            - name: energy_source_level_1
              description: Type of energy source (e.g. Renewable energy)
              type: string
            - name: energy_source_level_2
              description: Type of energy source (e.g. Wind, Solar)
              type: string
              opsd-contentfilter: "true"
            - name: technology
              description: Technology to harvest energy source (e.g. Onshore, Photovoltaics)
              type: string
            - name: electrical_capacity
              description: Installed electrical capacity in MW
              type: number
              format: float
            - name: dso
              description: Name of distribution system operator of the region the plant is located in
              type: string
            - name: gsrn_id
              description: Danish wind turbine identifier number (GSRN)
              type: number
              format: integer
            - name: postcode
              description: Danish zip-code
              type: string
            - name: municipality_code
              description: Danish 3-digit Kommune-Nr
              type: string
            - name: municipality
              description: Name of Danish Kommune
              type: string
            - name: address
              description: Street name or name of land parcel
              type: string
            - name: address_number
              description: House number or number of land parcel
              type: string
            - name: utm_east
              description: Coordinate in Universal Transverse Mercator (east)
              type: numeric
              format: float
            - name: utm_north
              description: Coordinate in Universal Transverse Mercator (north)
              type: numeric
              format: float
            - name: lat
              description: Latitude coordinates
              type: geopoint
              format: lat
            - name: lon
              description: Longitude coordinates 
              type: geopoint
              format: lon
            - name: hub_height
              description: Wind turbine hub heigth in m
              type: numeric
              format: float
            - name: rotor_diameter
              description: Wind turbine rotor diameter in m
              type: numeric
              format: float
            - name: manufacturer
              description: Company that has built the wind turbine
              type: string
            - name: model
              description: Wind turbind model type
              type: string
            - name: data_source
              description: Source of database entry
              type: string
    - path: renewable_power_plants_FR.csv
      format: csv
      encoding: UTF-8
      missingValue: ""
      schema:
          fields:
            - name: municipality_code
              description: French 5-digit INSEE code for Communes
              type: string
            - name: municipality
              description: Name of French Commune
              type: string
            - name: energy_source_level_1
              description: Type of energy source (e.g. Renewable energy)
              type: string
            - name: energy_source_level_2
              description: Type of energy source (e.g. Wind, Solar)
              type: string
              opsd-contentfilter: "true"
            - name: energy_source_level_3
              description: Subtype of energy source (e.g. Biomass and biogas)
              type: string
            - name: technology
              description: Technology to harvest energy source (e.g. Onshore, Photovoltaics)
              type: string
            - name: electrical_capacity
              description: Installed electrical capacity in MW
              type: number
              format: float
            - name: number_of_installations
              description: Number of installations of the energy source subtype in the municipality
              type: number
              format: integer
            - name: lat
              description: Latitude coordinates
              type: geopoint
              format: lat
            - name: lon
              description: Longitude coordinates 
              type: geopoint
              format: lon
            - name: data_source
              description: Source of database entry
              type: string
    - path: renewable_power_plants_PL.csv
      format: csv
      encoding: UTF-8
      missingValue: ""
      schema:
          fields:
            - name: district
              description: Name of the Polish powiat
              type: string
            - name: energy_source_level_1
              description: Type of energy source (e.g. Renewable energy)
              type: string
            - name: energy_source_level_2
              description: Type of energy source (e.g. Wind, Solar)
              opsd-contentfilter: "true"
              type: string
              opsd-contentfilter: "true"
            - name: energy_source_level_3
              description: Subtype of energy source (e.g. Biomass and biogas)
              type: string
            - name: technology
              description: Technology to harvest energy source (e.g. Onshore, Photovoltaics)
              type: string
            - name: electrical_capacity
              description: Installed electrical capacity in MW
              type: number
              format: float
            - name: number_of_installations
              description: Number of installations of the energy source subtype in the district
              type: number
              format: integer
            - name: data_source
              description: Source of database entry
              type: string
    - path: renewable_power_plants_UK.csv
      format: csv
      encoding: UTF-8
      missingValues: ""
      schema:
          fields:
            - name: commissioning_date
              type: date
              format: YYYY-MM-DD
              description: Date of commissioning of specific unit
              opsd-contentfilter: "true"
            - name: uk_beis_id
              type: number
              format: integer
            - name: site_name
              type: string
            - name: operator
              type: string
            - name: energy_source_level_1
              description: Type of energy source (e.g. Renewable energy)
              type: string
            - name: energy_source_level_2
              description: Type of energy source (e.g. Wind, Solar)
              opsd-contentfilter: "true"
              type: string
            - name: energy_source_level_3
              description: Type of energy source (e.g. Biomass and biogas)
              type: string
            - name: technology
              description: Technology to harvest energy source (e.g. Onshore, Photovoltaics)
              type: string
            - name: electrical_capacity
              description: Installed electrical capacity in MW
              type: number
              format: float
            - name: chp
              type: string
            - name: support_robranding
              type: number
              format: float
            - name: support_fit
              type: number
              format: float
            - name: support_cfd
              type: number
              format: float
            - name: capacity_individual_turbine
              type: number
              format: float
            - name: number_of_turbines
              type: number
              format: integer
            - name: solar_mounting_type
              type: string
            - name: address
              type: string
            - name: municipality
              type: string
            - name: region
              type: string
            - name: country
              description: The UK's constituent country in which the facility is located.
              type: string
            - name: postcode
              type: string
            - name: lat
              description: Latitude coordinates
              type: string
            - name: lon
              description: Longitude coordinates
              type: string
            - name: data_source
              description: The source of database entries
              type: string
            - name: comment
              description: Shortcodes for comments related to this entry, explanation can be looked up in validation_marker.csv
              type: string
    - path: renewable_power_plants_CH.csv
      format: csv
      encoding: UTF-8
      missingValue: ""
      schema:
          fields:
            - name: commissioning_date
              type: date
              format: YYYY-MM-DD
              opsd-contentfilter: "true"
            - name: municipality
              type: string
            - name: energy_source_level_1
              description: Type of energy source (e.g. Renewable energy)
              type: string
            - name: energy_source_level_2
              description: Type of energy source (e.g. Wind, Solar)
              type: string
              opsd-contentfilter: "true"
            - name: technology
              description: Technology to harvest energy source (e.g. Onshore, Photovoltaics)
              type: string
            - name: electrical_capacity
              description: Installed electrical capacity in MW
              type: number
              format: float
            - name: municipality_code
              type: number
              format: integer
            - name: project_name
              description: name of the project
              type: string
            - name: production
              description: yearly production in MWh
              type: numeric
              format: float
            - name: tariff
              description: tariff in CHF for 2016
              type: numeric
              format: float
            - name: notification_date
              description: date of data entriy at BFE
              type: date
              format: YYYY-MM-DD HH:MM:SS.SSSSS
            - name: street
              description: Street name
              type: string
            - name: canton
              description: name of the cantones/ member states of the Swiss confederation
              type: string
            - name: company
              description: name of the company
              type: string
            - name: lat
              description: Latitude coordinates
              type: geopoint
              format: lat
            - name: lon
              description: Longitude coordinates 
              type: geopoint
              format: lon
            - name: data_source
              description: Source of database entry
              type: string
    - path: res_plants_separated_DE_outvalidated_plants.csv
      format: csv
      encoding: UTF-8
      missingValue: ""
      schema:         
          fields:
            - name: commissioning_date
              type: date
              format: YYYY-MM-DD
              description: Date of commissioning of specific unit
            - name: decommissioning_date
              type: date
              format: YYYY-MM-DD
              description: Date of decommissioning of specific unit
            - name: energy_source_level_1
              description: Type of energy source (e.g. Renewable energy)
              type: string
            - name: energy_source_level_2
              description: Type of energy source (e.g. Wind, Solar)
              type: string
              opsd-contentfilter: "true"
            - name: energy_source_level_3
              description: Subtype of energy source (e.g. Biomass and biogas)
              type: string
            - name: technology
              description: Technology to harvest energy source (e.g. Onshore, Photovoltaics)
              type: string
            - name: electrical_capacity
              description: Installed electrical capacity in MW
              type: number
              format: float
              unit: MW
            - name: thermal_capacity
              description: Installed thermal capacity in MW
              type: number
              format: float
              unit: MW
            - name: voltage_level
              description: Voltage level of grid connection
              type: string
            - name: tso
              description: Name of transmission system operator of the area the plant is located
              type: string
            - name: dso
              description: Name of distribution system operator of the region the plant is located in
              type: string
            - name: dso_id
              description: Company number of German distribution grid operator
              type: string
            - name: eeg_id
              description: Power plant EEG (German feed-in tariff law) remuneration number
              type: string
            - name: bnetza_id
              description: Power plant identification number by BNetzA
              type: string
            - name: federal_state
              description: Name of German administrative level 'Bundesland'
              type: string
            - name: postcode
              description: German zip-code
              type: string
            - name: municipality_code
              description: German Gemeindenummer (municipalitiy number)
              type: string
            - name: municipality
              description: Name of German Gemeinde (municipality)
              type: string
            - name: address
              description: Street name or name of land parcel
              type: string
            - name: address_number
              description: House number or number of land parcel
              type: string
            - name: utm_zone
              description: Universal Transverse Mercator zone value
              type: 
            - name: utm_east
              description: Coordinate in Universal Transverse Mercator (east)
              type: numeric
              format: float
            - name: utm_north
              description: Coordinate in Universal Transverse Mercator (north)
              type: numeric
              format: float
            - name: lat
              description: Latitude coordinates
              type: geopoint
              format: lat
            - name: lon
              description: Longitude coordinates 
              type: geopoint
              format: lon
            - name: data_source
              description: Source of database entry
              type: string
            - name: comment
              description: Shortcodes for comments related to this entry, explanation can be looked up in validation_marker.csv
              type: string
    - path: res_plants_separated_FR_overseas_territories.csv
      format: csv
      encoding: UTF-8
      missingValue: ""
      schema:
          fields:
            - name: municipality_code
              description: French 5-digit INSEE code for Communes
              type: string
            - name: municipality
              description: Name of French Commune
              type: string
            - name: energy_source_level_1
              description: Type of energy source (e.g. Renewable energy)
              type: string
            - name: energy_source_level_2
              description: Type of energy source (e.g. Wind, Solar)
              type: string
              opsd-contentfilter: "true"
            - name: energy_source_level_3
              description: Subtype of energy source (e.g. Biomass and biogas)
              type: string
            - name: technology
              description: Technology to harvest energy source (e.g. Onshore, Photovoltaics)
              type: string
            - name: electrical_capacity
              description: Installed electrical capacity in MW
              type: number
              format: float
            - name: number_of_installations
              description: Number of installations of the energy source subtype in the municipality
              type: number
              format: integer
            - name: lat
              description: Latitude coordinates
              type: geopoint
              format: lat
            - name: lon
              description: Longitude coordinates 
              type: geopoint
              format: lon
            - name: data_source
              description: Source of database entry
              type: string
    - path: renewable_power_plants.xlsx
      format: xlsx
    - path: validation_marker.csv
      format: csv
      encoding: UTF-8
      mediatype: text/csv
      missingValue: ""
      schema:         
          fields:
            - name: Validation_Marker
              description: Name of validation marker utilized in column comment in the renewable_power_plant_germany.csv
              type: string
            - name: Explanation
              description: Comment explaining meaning of validation marker
              type: string
    - path: renewable_capacity_timeseries_DE.csv
      format: csv
      encoding: UTF-8
      mediatype: text/csv
      missingValue: ""
      schema:         
          fields:
            - name: day
              description: Day
              type: date
              format: YYYY-MM-DD
              opsd-contentfilter: "true"
            - name: Solar
              description: Cumulated electrical solar capacity in MW
              type: number
              format: float
              unit: MW
            - name: Onshore
              description: Cumulated electrical wind onshore capacity in MW
              type: number
              format: float
              unit: MW
            - name: Offshore
              description: Cumulated electrical wind offshore capacity in MW
              type: number
              format: float
              unit: MW
            - name: Bioenergy
              description: Cumulated electrical bioenergy and renewable waste capacity in MW
              type: number
              format: float
              unit: MW
            - name: Geothermal
              description: Cumulated electrical geothermal capacity in MW
              type: number  
              format: float
              unit: MW
            - name: Run-of-river
              description: Cumulated electrical run-of-river capacity in MW
              type: number 
              format: float
              unit: MW
    - path: renewable_capacity_timeseries_UK.csv
      format: csv
      encoding: UTF-8
      mediatype: text/csv
      missingValue: ""
      schema: 
          fields:
            - name: day
              description: Day
              type: date
              format: YYYY-MM-DD
              opsd-contentfilter: "true"
            - name: Hydro-Other or unspecified technology
              description: Cumulated electrical hydro capacity for unspecified technology in MW
              type: number
              format: float
              unit: MW
            - name: Hydro-Pumped storage
              description: Cumulated electrical pupmed-storage hydro capacity in MW
              type: number
              format: float
              unit: MW
            - name: Bioenergy
              description: Cumulated electrical bioenergy capacity in MW
              type: number
              format: float
              unit: MW
            - name: Onshore
              description: Cumulated electrical wind onshore capacity in MW
              type: number
              format: float
              unit: MW
            - name: Offshore
              description: Cumulated electrical wind offshore capacity in MW
              type: number
              format: float
              unit: MW
            - name: Marine
              description: Cumulated electrical marine capacity in MW
              type: number
              format: float
              unit: MW
            - name: Solar
              description: Cumulated electrical solar capacity in MW
              type: number
              format: float
              unit: MW
sources:
    - name: BNetzA
      web: http://www.bundesnetzagentur.de/cln_1422/DE/Sachgebiete/ElektrizitaetundGas/Unternehmen_Institutionen/ErneuerbareEnergien/Anlagenregister/Anlagenregister_Veroeffentlichung/Anlagenregister_Veroeffentlichungen_node.html
      source: Bundesnetzagentur register of renewable power plants (excl. PV)
    - name: BNetzA_PV
      web: http://www.bundesnetzagentur.de/cln_1431/DE/Sachgebiete/ElektrizitaetundGas/Unternehmen_Institutionen/ErneuerbareEnergien/Photovoltaik/DatenMeldgn_EEG-VergSaetze/DatenMeldgn_EEG-VergSaetze_node.html    
      source: Bundesnetzagentur register of PV power plants
    - name: TransnetBW, TenneT, Amprion, 50Hertz, Netztransparenz.de
      web: https://www.netztransparenz.de/de/Anlagenstammdaten.htm
      source: Netztransparenz.de - information platform of German TSOs (register of renewable power plants in their control area)
    - name: Postleitzahlen Deutschland
      web: http://www.suche-postleitzahl.org/downloads
      source: Zip codes of Germany linked to geo-information
    - name: Energinet.dk
      web: http://www.energinet.dk/SiteCollectionDocuments/Danske%20dokumenter/El/SolcelleGraf.xlsx
      source: register of Danish wind power plants
    - name: Energistyrelsen
      web: http://www.ens.dk/sites/ens.dk/files/info/tal-kort/statistik-noegletal/oversigt-energisektoren/stamdataregister-vindmoeller/anlaegprodtilnettet.xls    
      source: ens.dk - register of Danish PV power plants
    - name: GeoNames
      web: http://download.geonames.org/export/zip/
      source: geonames.org
    - name: Ministry for the Ecological and Inclusive Transition
      web: https://www.statistiques.developpement-durable.gouv.fr/donnees-locales-relatives-aux-installations-de-production-delectricite-renouvelable-beneficiant-0?rubrique=23&dossier=189
    - name: OpenDataSoft
      web: http://public.opendatasoft.com/explore/dataset/correspondance-code-insee-code-postal/download/'\
           '?format=csv&refine.statut=Commune%20simple&timezone=Europe/Berlin&use_labels_for_header=true
      source: Code Postal - Code INSEE
    - name: Urzad Regulacji Energetyki (URE)
      web: http://www.ure.gov.pl/uremapoze/mapa.html
      source: Energy Regulatory Office of Poland
    - name: Bundesamt für Energie (BFE)
      web: http://www.bfe.admin.ch/themen/00612/02073/index.html?dossier_id=02166&lang=de
      source: Swiss Federal Office of Energy
    - name: UK Government Department of Business, Energy & Industrial Strategy (BEIS)
      web: https://www.gov.uk/government/publications/renewable-energy-planning-database-monthly-extract
      source: Renewable Energy Planning Database quarterly extract
contributors:
    - name: Ingmar Schlecht
      email: schlecht@neon-energie.de
    - name: Milos Simic
      email: milos.simic.ms@gmail.com
"""

metadata = yaml.load(metadata)


metadata['id'] = 'https://doi.org/10.25832/renewable_power_plants/'+settings['version']
metadata['last_changes'] = settings['changes']
metadata['version'] = settings['version']

metadata['documentation'] = 'https://github.com/Open-Power-System-Data/renewable_power_plants/blob/'+settings['version']+'/main.ipynb'

datapackage_json = json.dumps(metadata, indent=4, separators=(',', ': '))

# Write the information of the metadata
with open(os.path.join(package_path, 'datapackage.json'), 'w') as f:
    f.write(datapackage_json)

## Generate checksums

Generates checksums.txt

In [ ]:
def get_sha_hash(path, blocksize=65536):
    sha_hasher = hashlib.sha256()
    with open(path, 'rb') as f:
        buffer = f.read(blocksize)
        while len(buffer) > 0:
            sha_hasher.update(buffer)
            buffer = f.read(blocksize)
        return sha_hasher.hexdigest()

files = [
    'validation_marker.csv', 'renewable_capacity_timeseries_DE.csv',
    'renewable_power_plants.sqlite', 'renewable_power_plants.xlsx',
]

for country in countries_including_dirty:
    files.append(table_names[country]+'.csv')

for country in daily_timeseries:
    files.append('renewable_capacity_timeseries_{}.csv'.format(country))
    
with open('checksums.txt', 'w') as f:
    for file_name in sorted(files):
        file_hash = get_sha_hash(os.path.join(package_path, file_name))
        f.write('{},{}\n'.format(file_name, file_hash))